In [0]:
from pyspark.sql import SparkSession

In [0]:
data = [("Anurag", "IT", 50000), ("Stuti", "HR", 45000), ("Vaishnav", "IT", 60000)]
cols = ["Name", "Department", "Salary"]

df = spark.createDataFrame(data,cols)
display(df)

In [0]:

# save i a table so that data is not lost whn compute engine is stopped or restarted
df.write.format("delta").mode("overwrite").saveAsTable("employee_delta")

In [0]:
%sql
select * from employee_delta

In [0]:
# to see the versions of employee delta table
%sql
describe history employee_delta

In [0]:
# update table with new values -doesn't work with comments - remove this line and then run the cell
%sql
update employee_delta set Salary = 100000 where Name = 'Stuti'

In [0]:
%sql 
describe history employee_delta

In [0]:
# MERGE INTO - sql cmd where we can update or insert rows if ot present -prevents duplication

from pyspark.sql import Row

update_data = [Row(Name="Alice", Department="IT", Salary=52000)]
df_update = spark.createDataFrame(update_data)
df_update.createOrReplaceTempView("updates_view")


spark.sql("""
MERGE INTO employee_delta AS target
USING updates_view AS source
ON target.Name = source.Name
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *
""")

In [0]:
%sql
select * from employee_delta

In [0]:
#Schema Evolution (Add New Column)

new_data = [("David", "Finance", 70000, 5)]  # 5 years experience
cols_new = ["Name", "Department", "Salary", "Experience"]
df_new=spark.createDataFrame(new_data,cols_new) 
display(df_new)


#Merge Schema 
df_new.write.format("delta") \
    .option("mergeSchema", "true") \
    .mode("append") \
    .saveAsTable("employees_delta")

In [0]:
%sql
select * from employees_delta

In [0]:
%sql
describe extended employees_delta

In [0]:
%sql
describe history employee_delta